In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [ ]:
amaz = pd.read_csv("/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv")
amaz.head()

In [ ]:
amaz.info()

We are assuming the price of Books in USD as amazon is HQ'd at USA.

In [ ]:
amaz.rename(columns={'Price':'Price_USD'},inplace=True)

In [ ]:
amaz.isna().sum()

In [ ]:
amaz.duplicated().sum()

Seems as if our data is free from missing value and also it doesn't contain any duplicate values.

In [ ]:
nf = [u for u in amaz.columns if amaz[u].dtypes != 'O']
cf = [u for u in amaz.columns if u not in nf]
print("\nNumerical Features in our dataset are: ",nf,"\n")
print("Character Features in our dataset are: ",cf)

## Going through the Numerical features

In [ ]:
amaz[nf].nunique()

Out of our numerical features Year is an Temporal Feature.<br>
Price and Reviews being the Continuous Feature.<br>
User Rating being Discrete Numeric Feature.

### Univariate Analysis.

In [ ]:
amaz.describe()

In [ ]:
plt.subplots(2,2,figsize=(14,10))

plt.subplot(2,2,1)
sns.boxplot(data = amaz['User Rating'],color='lightgreen')
plt.title("User Ratings")

plt.subplot(2,2,2)
sns.boxplot(data = amaz['Reviews'],color='pink')
plt.title("Reviews")

plt.subplot(2,2,3)
sns.boxplot(data = amaz['Price_USD'],color='yellow')
plt.title("Price in USD")

plt.subplot(2,2,4)
sns.boxplot(data = amaz['Year'],color='lightblue')
plt.title('Year')

plt.show()

In [ ]:
amaz.Year.value_counts()

**From the dataset we get to know that:**<br>
  * The books have been rated pretty good with the median value nearing 4.7
  * On an average we are getting about 9-10k reviews on the books,and max of nearly 80k reviews.
  * Median price of books is near 17 USD, and **we do have books priced at 0** probably some error
  * We have same count of books i.e.50 from each year.

In [ ]:
plt.subplots(1,3,figsize=(14,3))

for i in nf[:3]:
    plt.subplot(1,3,nf.index(i)+1)
    print(i,sep='\t')
    sns.histplot(amaz[i])

plt.show()

All the data present is skewed.<br> **User Rating** and **Price feature** being **Right Skewed** and **Reviews** being **Left Skewed**.

### Bi-Variate Analysis

**Here we have the correlation Table.**

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(amaz.corr(),annot=True)
plt.show()

**There is no sign of any  kind of correlation among the features.<br>All the features are poorly correlated.**

In [ ]:
sns.pairplot(amaz)
plt.show()

Lets have a deeper look on these graphs.

In [ ]:
px.scatter(data_frame=amaz,x='Reviews',y='User Rating',hover_name='Author',color='Genre')

In [ ]:
px.scatter(data_frame=amaz,x='Reviews',y='Price_USD',hover_name='Name',color='Genre')

In [ ]:
print(f"Most costly Book: {amaz[amaz.Price_USD == amaz.Price_USD.max()].Name.to_list()[0]}")

In [ ]:
px.scatter(data_frame=amaz,x='Price_USD',y='User Rating',hover_name='Author',color='Genre')

In [ ]:
amaz[amaz.Author == 'Dav Pilkey']['Name'].to_list()[0]

## For Character Featues

In [ ]:
amaz[cf].nunique()

So, we have 2 types of Genre's.<br>
248 Authors and
**351** Books

**Wait whatttt!! 351 books but our dataset had 550 entries. Right ?**<br>
Let's inspect

In [ ]:
amaz.describe(include='O')

In [ ]:
amaz[cf].head()

They all seem to be of Nominal Category.

In [ ]:
amaz[cf].duplicated().sum()

Okay, so initially when we queired about the duplicate values we got none<br>
but now we are getting 199.

In [ ]:
amaz[['Name', 'Author', 'User Rating', 'Reviews', 'Price_USD','Genre']].duplicated().sum()


# ds = amaz[amaz[['Name', 'Author', 'User Rating', 'Reviews', 'Price_USD','Genre']].duplicated()]
# amaz.drop(ds.index,inplace=True)

In [ ]:
amaz[amaz[cf].duplicated()]

From a bit of exploration, I have get this random book. Lemme check about this book.

In [ ]:
amaz[amaz['Name']== 'The Help']

So, here are some entries with same book name and different years<br> Okay!!

Now lets get back to where we were.

In [ ]:
sns.countplot(x='Genre',data = amaz)
plt.show()

As per results we have more number of Non-fictional books

**Top 15 authors with respect to count books in best sellers**

In [ ]:
plt.figure(figsize=(12,6))
amaz.Author.value_counts(ascending=True)[233:].plot.barh(color='violet')
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
b = amaz.Name.value_counts(ascending =True)[341:]
b.plot.barh(color='green')
plt.show()

Okay, so this chart above tells us that out of the data of **11** total years **(right from 2009 to 2019)** we have followig books being the top seller for above mentioned tears.

In [ ]:
amaz[amaz.Name == b.index[-1]]

In [ ]:
for i in range(len(b)-1,6,-1):
    print(f"With {b.index[i]} being best seller for {b[i]} years.\n")
print("& so on.")

In [ ]:
sns.barplot(x='Genre',y='Price_USD', data = amaz, estimator = np.median)
plt.show()

This can be interpretd as Books of Non-Fiction Genre actually tend to be costly than the Fictional Ones.

In [ ]:
sns.barplot(x='Genre',y='Reviews', data = amaz, estimator = np.median)
plt.show()

Fictional books have a bigger customer base than the Non-Fictional ones.

In [ ]:
u = amaz.groupby('Genre')['Author'].value_counts()

### Best Sellers in each Genre

In [ ]:
u = u.to_frame()
u = u.rename(columns={'Author':'A_Count'}).reset_index()
# u[u.Genre=='Fiction'].head(10).plot.barh(x='Author')
# fig = px.bar(data_frame=u[u.Genre=='Fiction'].head(10),x='Author',y='A_Count')
# fig.show()

In [ ]:
fig1 = px.bar(data_frame=u[u.Genre=='Fiction'].head(10),x='Author',y='A_Count',color='Genre')
fig2 = px.bar(data_frame=u[u.Genre=='Non Fiction'].head(10),x='Author',y='A_Count',color='Genre',color_discrete_sequence=["goldenrod"])
fig1.show()
fig2.show()

In [ ]:
t = u[u.Genre=='Fiction'].iloc[:3,1]
print(f"So {t.to_list()} are the top authors of Fiction Genre.")
print("&")
t = u[u.Genre=='Non Fiction'].iloc[:3,1]
print(f"So {t.to_list()} are the top authors of Fiction Genre.")

### Best Sellers by year with respect to Genre

In [ ]:
yg = amaz.groupby('Year')['Genre'].value_counts().to_frame().rename(columns={'Genre':'Books'}).reset_index()

fig = px.line(data_frame=yg,x='Year',y='Books',color='Genre',hover_name='Genre')
fig.update_traces(mode='lines+markers')
fig.show()

In [ ]:
# plt.figure(figsize=(12,5))
ygp = amaz.groupby(['Year','Genre'])['Price_USD'].median().to_frame().reset_index()
fig = px.line(data_frame=ygp,x='Year',y='Price_USD',color='Genre')
fig.update_traces(mode='lines+markers')
# .plot.line()
fig.show()

In [ ]:
ur = amaz.groupby('User Rating')['Genre'].value_counts().to_frame().rename(columns={'Genre':'Books Count'}).reset_index()

fig = px.bar(data_frame=ur,x='User Rating',y='Books Count',color='Genre',hover_name='Genre')
# fig.update_traces(mode='lines+markers')
fig.show()

**Top 10 Best authors in their Respective Genre's**

In [ ]:
ag = amaz.groupby(['Author','Genre'])['User Rating'].median().sort_values().to_frame().reset_index()

agf = ag[ag.Genre=='Fiction'].sort_values(by='User Rating',ascending=False).head(10)
agn = ag[ag.Genre=='Non Fiction'].sort_values(by='User Rating',ascending=False).head(10)

fig1 = px.bar(data_frame=agf,y='User Rating',x='Author',color='Genre')
fig2 = px.bar(data_frame=agf,y='User Rating',x='Author',color='Genre',color_discrete_sequence=["green"])
fig1.show()
fig2.show()


**Top authors with respect to reviews**

In [ ]:
plt.figure(figsize=(14,7))
amaz.groupby(['Author'])['Reviews'].sum().sort_values(ascending = True)[228:].plot.barh()
plt.xticks(rotation=0)
plt.show()

## Observations:


 - There's no correlation among the features.
 - All the numeric features are skewed.
 - **Delia Owens** seems to be most read as well as best rated Author.
 - **Diagnostic and Statistical Manual of Mental Disorders, 5th Edition: DSM-5** is the most costly book among the Best-Sellers
 - **Dave Pilkey's** book **(Dog Man...)** has been cheapest as well as the best rated among the Best-Sellers.
 - **Publication Manual of the American Psychological Association, 6th Edition** Book has remained 10 out of 11 times as the Best Seller.
 - Among the 50 best sellers there are more number of **Non-Fictional** books than the Fictional ones.
 - Author **Jeff Kinney**'s Books has made it to Top 50 most number of times.
 - It has been observed that the **Non-Fictional** books are costlier than the Fictional ones.
 - **Fictional** books have a bigger customer base than the Non-Fictional ones.
 - In the **Fiction** Genre, Jeff Kinney's book while in **Non-Fiction** Genre, Gary Chapman has been Best Seller max number of times.
 - Every Year except 2014, most number of Books are of Non Fictional Genre.
 - Non-Fictional Books making to the Top-50 Best Sellers have min rating of 4.0. while in case of Fictional ones there is a lot of randomness with min rating of 3.3.
 - Best Rated Authors in:
     - **Fictional Genre:** Jill Twiss, Patrick Thorpe, Dr. Seuss, Rush Limbaugh, Emily Winfield Martin, Nathan W. Pyle, Eric Carle, Alice Schertle, Sherri Duskey Rinker, Dav Pilkey, Bill Martin Jr
     - **Non-Fictional Genre**: Chip Gaines,Sarah Young,Pete Souza,Lin-Manuel Miranda, Mark R. Levin, Brandon Stanton, Hopscotch Girls, Anthony Bourdain, Ron Chernow, Stephen Kendrick, Bessel van der Kolk M.D.<br>
     
  - Among all the Authors Suzanne Collins's book has been read by maximum no. of people.

**If you have any suggestions please comment down below.
I am learning about EDA and i'll definitely try to improve as per your suggestions.**